# 2-1でクロールした勝ち式のデータを回収率シミュレーションに使える形に成形

## 1. データをロードして統合

In [1]:
import pandas as pd
import glob
import os
win_and_pay_df_list = [pd.read_pickle(file) for file in glob.glob(os.path.join('../../data/crawledData/win_and_pay', '*.pkl'))]
win_and_pay_df = pd.concat(win_and_pay_df_list)
win_and_pay_df

3連単  3連単odds        3連複 3連複odds     2連単  \
date       venue raceNumber                                                  
2020-11-03 桐　生   1           [1, 4, 3]   ¥3,540  [1, 3, 4]  ¥1,000  [1, 4]   
                 2           [1, 5, 3]   ¥1,610  [1, 3, 5]    ¥470  [1, 5]   
                 3           [1, 3, 4]     ¥660  [1, 3, 4]    ¥320  [1, 3]   
                 4           [2, 3, 1]   ¥9,270  [1, 2, 3]    ¥320  [2, 3]   
                 5           [1, 3, 6]   ¥6,400  [1, 3, 6]  ¥4,000  [1, 3]   
...                                ...      ...        ...     ...     ...   
2020-11-26 福　岡   8           [1, 4, 5]   ¥1,520  [1, 4, 5]    ¥860  [1, 4]   
                 9           [4, 2, 1]   ¥4,590  [1, 2, 4]    ¥320  [4, 2]   
                 10          [1, 6, 4]   ¥5,120  [1, 4, 6]    ¥540  [1, 6]   
                 11          [3, 4, 2]  ¥43,480  [2, 3, 4]  ¥6,300  [3, 4]   
                 12          [2, 4, 3]  ¥21,820  [2, 3, 4]  ¥2,450  [2, 4]   

                             2連単odds     2連複 2連複odds   拡連複_1 拡連複odds_1  \
date       venue raceNumber                                              
2020-11-03 桐　生   1              ¥960  [1, 4]    ¥950  [1, 4]      ¥450   
                 2              ¥580  [1, 5]    ¥600  [1, 5]      ¥190   
                 3              ¥310  [1, 3]    ¥250  [1, 3]      ¥120   
                 4            ¥3,710  [2, 3]  ¥1,590  [2, 3]      ¥180   
                 5              ¥800  [1, 3]    ¥670  [1, 3]      ¥290   
...                              ...     ...     ...     ...       ...   
2020-11-26 福　岡   8              ¥440  [1, 4]    ¥460  [1, 4]      ¥190   
                 9            ¥1,440  [2, 4]    ¥570  [2, 4]      ¥200   
                 10           ¥1,870  [1, 6]  ¥1,480  [1, 6]      ¥470   
                 11          ¥11,890  [3, 4]  ¥3,570  [3, 4]      ¥930   
                 12           ¥7,180  [2, 4]  ¥2,340  [2, 4]      ¥990   

                              拡連複_2 拡連複odds_2   拡連複_3 拡連複odds_3 単勝  単勝odds  \
date       venue raceNumber                                                  
2020-11-03 桐　生   1           [1, 3]      ¥200  [3, 4]      ¥420  1    ¥190   
                 2           [1, 3]      ¥130  [3, 5]      ¥390  1    ¥110   
                 3           [1, 4]      ¥140  [3, 4]      ¥180  1    ¥130   
                 4           [1, 2]      ¥170  [1, 3]      ¥160  2  ¥1,290   
                 5           [1, 6]      ¥450  [3, 6]    ¥1,840  1    ¥140   
...                             ...       ...     ...       ... ..     ...   
2020-11-26 福　岡   8           [1, 5]      ¥210  [4, 5]      ¥660  1    ¥100   
                 9           [1, 4]      ¥280  [1, 2]      ¥140  4    ¥590   
                 10          [1, 4]      ¥210  [4, 6]      ¥470  1    ¥140   
                 11          [2, 3]      ¥640  [2, 4]      ¥450  3    ¥640   
                 12          [2, 3]      ¥670  [3, 4]      ¥250  2    ¥760   

                            複勝_1 複勝odds_1 複勝_2 複勝odds_2  
date       venue raceNumber                              
2020-11-03 桐　生   1             1     ¥120    4     ¥360  
                 2             1     ¥150    5     ¥300  
                 3             1     ¥210    3     ¥380  
                 4             2     ¥360    3     ¥160  
                 5             1     ¥130    3     ¥660  
...                          ...      ...  ...      ...  
2020-11-26 福　岡   8             1     ¥100    4     ¥180  
                 9             4     ¥120    2     ¥110  
                 10            1     ¥150    6     ¥660  
                 11            3     ¥590    4   ¥1,450  
                 12            2     ¥580    4     ¥580  

[28181 rows x 20 columns]

## 2. 使いやすい形に成形
- 連系の勝ち式は、X-Y-Zの形の文字列に
- 払戻し倍率はint型に

In [9]:
import re
from datetime import datetime as dt

def make_odds_to_int(str):
    
    out_int = int("".join(re.split("[¥,]", str)))
    
    return out_int

# 一応コピー
win_and_pay_df_arranged = win_and_pay_df.copy()
# Nanを含む列を削除
win_and_pay_df_arranged = win_and_pay_df_arranged[~(win_and_pay_df_arranged["拡連複odds_1"]=="\xa0")]
win_and_pay_df_arranged = win_and_pay_df_arranged[~(win_and_pay_df_arranged["拡連複odds_2"]=="\xa0")]
win_and_pay_df_arranged = win_and_pay_df_arranged[~(win_and_pay_df_arranged["拡連複odds_3"]=="\xa0")]
win_and_pay_df_arranged = win_and_pay_df_arranged[~(win_and_pay_df_arranged["複勝odds_1"]=="\xa0")]
win_and_pay_df_arranged = win_and_pay_df_arranged[~(win_and_pay_df_arranged["複勝odds_2"]=="\xa0")]

# odds関連の列名のリスト
odds_column_list = ["3連単odds", "3連複odds", "2連単odds", "2連複odds",
                    "拡連複odds_1", "拡連複odds_2", "拡連複odds_3",
                    "単勝odds", "複勝odds_1", "複勝odds_2"
                   ]

# oddds関連の列をint型に
for odds_column in odds_column_list:
    win_and_pay_df_arranged[odds_column] = win_and_pay_df_arranged[odds_column
                                                                  ].map(lambda x: make_odds_to_int(x))
# 勝式関連の列名のリスト
win_number_column_list = ["3連単", "3連複", "2連単", "2連複",
                    "拡連複_1", "拡連複_2", "拡連複_3",
                    "単勝", "複勝_1", "複勝_2"]
  
# 枠番の列をstr型に
for win_number in win_number_column_list:
    win_and_pay_df_arranged[win_number] = win_and_pay_df_arranged[win_number
                                                                  ].map(lambda x: "-".join(x))

# index部分のデータ型を整える
win_and_pay_df_arranged = win_and_pay_df_arranged.reset_index()
# dateをdatetime型に
win_and_pay_df_arranged["date"] = win_and_pay_df_arranged["date"].map(lambda x: dt.strptime(x, '%Y-%m-%d'))
# raceNumberはfloatに
win_and_pay_df_arranged["raceNumber"] = win_and_pay_df_arranged["raceNumber"].map(lambda x: float(x))
win_and_pay_df_arranged = win_and_pay_df_arranged.set_index(["date", "venue", "raceNumber"])

win_and_pay_df_arranged

3連単  3連単odds    3連複  3連複odds  2連単  2連単odds  \
date       venue raceNumber                                                 
2020-11-03 桐　生   1.0         1-4-3     3540  1-3-4     1000  1-4      960   
                 2.0         1-5-3     1610  1-3-5      470  1-5      580   
                 3.0         1-3-4      660  1-3-4      320  1-3      310   
                 4.0         2-3-1     9270  1-2-3      320  2-3     3710   
                 5.0         1-3-6     6400  1-3-6     4000  1-3      800   
...                            ...      ...    ...      ...  ...      ...   
2020-11-26 福　岡   8.0         1-4-5     1520  1-4-5      860  1-4      440   
                 9.0         4-2-1     4590  1-2-4      320  4-2     1440   
                 10.0        1-6-4     5120  1-4-6      540  1-6     1870   
                 11.0        3-4-2    43480  2-3-4     6300  3-4    11890   
                 12.0        2-4-3    21820  2-3-4     2450  2-4     7180   

                             2連複  2連複odds 拡連複_1  拡連複odds_1 拡連複_2  拡連複odds_2  \
date       venue raceNumber                                                   
2020-11-03 桐　生   1.0         1-4      950   1-4        450   1-3        200   
                 2.0         1-5      600   1-5        190   1-3        130   
                 3.0         1-3      250   1-3        120   1-4        140   
                 4.0         2-3     1590   2-3        180   1-2        170   
                 5.0         1-3      670   1-3        290   1-6        450   
...                          ...      ...   ...        ...   ...        ...   
2020-11-26 福　岡   8.0         1-4      460   1-4        190   1-5        210   
                 9.0         2-4      570   2-4        200   1-4        280   
                 10.0        1-6     1480   1-6        470   1-4        210   
                 11.0        3-4     3570   3-4        930   2-3        640   
                 12.0        2-4     2340   2-4        990   2-3        670   

                            拡連複_3  拡連複odds_3 単勝  単勝odds 複勝_1  複勝odds_1 複勝_2  \
date       venue raceNumber                                                   
2020-11-03 桐　生   1.0          3-4        420  1     190    1       120    4   
                 2.0          3-5        390  1     110    1       150    5   
                 3.0          3-4        180  1     130    1       210    3   
                 4.0          1-3        160  2    1290    2       360    3   
                 5.0          3-6       1840  1     140    1       130    3   
...                           ...        ... ..     ...  ...       ...  ...   
2020-11-26 福　岡   8.0          4-5        660  1     100    1       100    4   
                 9.0          1-2        140  4     590    4       120    2   
                 10.0         4-6        470  1     140    1       150    6   
                 11.0         2-4        450  3     640    3       590    4   
                 12.0         3-4        250  2     760    2       580    4   

                             複勝odds_2  
date       venue raceNumber            
2020-11-03 桐　生   1.0              360  
                 2.0              300  
                 3.0              380  
                 4.0              160  
                 5.0              660  
...                               ...  
2020-11-26 福　岡   8.0              180  
                 9.0              110  
                 10.0             660  
                 11.0            1450  
                 12.0             580  

[28040 rows x 20 columns]

In [10]:
# 予測用のdf: pickleファイルで保存
win_and_pay_df_arranged.to_pickle('../../data/arrangedData/win_and_pay_df.pkl')